In [1]:
import random
import numpy as np
import gymnasium as gym
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.autograd as autograd
from torch.autograd import Variable

In [ ]:
learning_rate = 5e-4 # alpha, 0.0005
mini_batch_size = 100 # number of observations used per step to update the model.
discount_factor = 0.9 # gamma - closer to one the more consideration of future rewards
replay_buffer_size = int(1e5) # For experience Replay, number of observations in AI memory, 100,000
interpolation_parameter = 1e-3 # 0.001

In [4]:
env = gym.make('FrozenLake-v1', desc=["SFFH","FFFH","HFFF","HFFG"], map_name="4x4", is_slippery=False, render_mode='human')

state_size = env.observation_space.n
action_size = env.action_space.n

In [3]:
class Network(nn.Module):
    def __init__(self, state_size, action_size, seed=42):
        super(Network, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.full_connected_layer_1 = nn.Linear(state_size,32)
        self.full_connected_layer_2 = nn.Linear(32,32)
        self.full_connected_layer_3 = nn.Linear(32,action_size)

    def forward(self,state):
        X = self.full_connected_layer_1(state)
        X = F.relu(x)
        X = self.full_connected_layer_2(x)
        X = F.relu(x)
        return self.full_connected_layer_3(x)

In [7]:
class ReplayMemory(object):
    def __init__(self, capacity):
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.capacity = capacity
        self.memory = []

    def push(self,event):
        self.memory.append(event)
        if len(self.memory) > self.capacity:
            del self.memory[0]

    def sample(self, batch_size):
        experiences = random.sample(self.memory, k=batch_size)
        states = torch.from_numpy(np.vstack([e[0] for e in experiences if e is not None])).float().to(self.device)
        actions = torch.from_numpy(np.vstack([e[0] for e in experiences if e is not None])).float().to(self.device)
        rewards = torch.from_numpy(np.vstack([e[0] for e in experiences if e is not None])).float().to(self.device)
        next_sates = torch.from_numpy(np.vstack([e[0] for e in experiences if e is not None])).float().to(self.device)
        terminated = torch.from_numpy(np.vstack([e[0] for e in experiences if e is not None])).float().to(self.device)
        return states, next_state, actions, rewards, terminated